In [57]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [58]:
PROBLEM_SIZE = 10
NUM_SETS = 30
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [59]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def cost(state):
    return len(state.taken)

def distance(state, action):

    taken_sets=list(state.taken)
    #print("taken sets", taken_sets)
    #print("action",action)
    currently_covered=reduce(
        np.logical_or,
        [SETS[i] for i in taken_sets],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    #print("currently_covered",currently_covered)
 
    if action:
        newly_covered=SETS[action]
        #print("newly_covered", newly_covered)
        difference=0
        for i in range(len(newly_covered)):
            if (newly_covered[i] == currently_covered[i] ):
                difference+=1
        #print("difference",difference)
        overlap=difference

    else:
        overlap=0
    
    new_state=[taken_sets,action]
    distance=PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in range(len(new_state))],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    #print("overlap",overlap)
    #print("distance",distance)
    return 0.3*overlap+0.7*distance

def f(state, action):
    return distance(state,action)+cost(state)

In [60]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [61]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state,None), state))

counter = 0
_, current_state = frontier.get()
#print("initial current state", current_state)
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(current_state,action), new_state))
        
    _, current_state = frontier.get()
    #print("current state",current_state)

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 34 steps (3 tiles)


In [62]:
current_state

State(taken={3, 21, 7}, not_taken={0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29})

In [63]:
for i in range(len(SETS)):
    print(i)
    print(SETS[i])

0
[False False False False False False False  True  True False]
1
[False False False False  True  True False False False  True]
2
[False  True  True False False False  True False  True False]
3
[False False False False False  True False  True False  True]
4
[False False False False  True False  True False False False]
5
[False False  True False False False False False False False]
6
[False False  True False False False False False False False]
7
[ True False False False False False  True False False False]
8
[False False  True False  True False  True False  True False]
9
[False  True  True False False False  True False False False]
10
[False False False  True  True  True False False False False]
11
[False False False False False  True False False  True False]
12
[ True False False False False  True False False False False]
13
[False False False False False False False  True False False]
14
[False False False False False  True  True  True  True False]
15
[ True  True False False False F